In [1]:
from openagentkit.modules.openai import OpenAIExecutor
from openagentkit.core.utils.tool_wrapper import tool
from pydantic import BaseModel
from typing import Annotated
import openai
import os

ModuleNotFoundError: No module named 'openagentkit'

In [ ]:
# Define a tool
@tool(
    description="Get the weather of a city", # Define the tool description
)
def get_weather(city: Annotated[str, "The city to get the weather of"]): # Each argument must be of type Annotated
    """Get the weather of a city"""

    # Actual implementation here...
    # ...

    return f"Weather in {city}: sunny, 20°C, feels like 22°C, humidity: 50%"

In [ ]:
# Initialize OpenAI client
client = openai.OpenAI(
    api_key=os.getenv("OPENAI_API_KEY"),
)

# Initialize LLM service
executor = OpenAIExecutor(
    client=client,
    model="gpt-4o-mini",
    system_message="""
    You are a helpful assistant that can answer questions and help with tasks.
    You are also able to use tools to get information.
    """,
    tools=[get_weather],
    temperature=0.5,
    max_tokens=100,
    top_p=1.0,
)

generator = executor.execute(
    messages=[
        {"role": "user", "content": "What's the weather like in New York?"}
    ]
)

for response in generator:
    print(response)

2025-04-18 00:09:30.875 | INFO     | openagentkit.modules.openai.openai_executor:execute:164 - Response Received: role='assistant' content=None tool_calls=[{'id': 'call_b21Mn2v1yHyjVqf6luXyBITn', 'type': 'function', 'function': {'arguments': '{"city":"New York"}', 'name': 'get_weather'}}] tool_results=None refusal=None audio=None usage=UsageResponse(prompt_tokens=112, completion_tokens=16, total_tokens=128, prompt_tokens_details=PromptTokensDetails(cached_tokens=0, audio_tokens=0), completion_tokens_details=CompletionTokensDetails(reasoning_tokens=0, audio_tokens=0, accepted_prediction_tokens=0, rejected_prediction_tokens=0))
2025-04-18 00:09:30.877 | INFO     | openagentkit.core.handlers.tool_handler:handle_tool_request:179 - Tool Result: Weather in New York: sunny, 20°C, feels like 22°C, humidity: 50%


role='assistant' content='None' tool_calls=[{'id': 'call_b21Mn2v1yHyjVqf6luXyBITn', 'type': 'function', 'function': {'arguments': '{"city":"New York"}', 'name': 'get_weather'}}] tool_results=None refusal=None audio=None usage=UsageResponse(prompt_tokens=112, completion_tokens=16, total_tokens=128, prompt_tokens_details=PromptTokensDetails(cached_tokens=0, audio_tokens=0), completion_tokens_details=CompletionTokensDetails(reasoning_tokens=0, audio_tokens=0, accepted_prediction_tokens=0, rejected_prediction_tokens=0))
role='tool' content='None' tool_calls=None tool_results=[ToolCallResult(tool_name='get_weather', result='Weather in New York: sunny, 20°C, feels like 22°C, humidity: 50%')] refusal=None audio=None usage=UsageResponse(prompt_tokens=112, completion_tokens=16, total_tokens=128, prompt_tokens_details=PromptTokensDetails(cached_tokens=0, audio_tokens=0), completion_tokens_details=CompletionTokensDetails(reasoning_tokens=0, audio_tokens=0, accepted_prediction_tokens=0, rejected_p

2025-04-18 00:09:32.404 | INFO     | openagentkit.modules.openai.openai_executor:execute:164 - Response Received: role='assistant' content='The weather in New York is currently sunny, with a temperature of 20°C (feels like 22°C) and a humidity level of 50%.' tool_calls=None tool_results=None refusal=None audio=None usage=UsageResponse(prompt_tokens=162, completion_tokens=34, total_tokens=196, prompt_tokens_details=PromptTokensDetails(cached_tokens=0, audio_tokens=0), completion_tokens_details=CompletionTokensDetails(reasoning_tokens=0, audio_tokens=0, accepted_prediction_tokens=0, rejected_prediction_tokens=0))


role='assistant' content='The weather in New York is currently sunny, with a temperature of 20°C (feels like 22°C) and a humidity level of 50%.' tool_calls=None tool_results=[] refusal=None audio=None usage=UsageResponse(prompt_tokens=162, completion_tokens=34, total_tokens=196, prompt_tokens_details=PromptTokensDetails(cached_tokens=0, audio_tokens=0), completion_tokens_details=CompletionTokensDetails(reasoning_tokens=0, audio_tokens=0, accepted_prediction_tokens=0, rejected_prediction_tokens=0))


In [ ]:
executor.clear_history()

[{'role': 'system',
  'content': '\n    You are a helpful assistant that can answer questions and help with tasks.\n    You are also able to use tools to get information.\n    \n        Current date and time: 2025-04-18 00:09:29\n\n        \n        '}]

In [ ]:
while True:
    user_input = input("User: ")
    print(f"User: {user_input}")
    
    if user_input.lower() == "exit":
        break

    generator = executor.execute(
        messages=[
            {"role": "user", "content": user_input}
        ]
    )

    for response in generator:
        if response.content is not None:
            print(f"AI: {response.content}")
        if response.tool_calls:
            [print(f"Tool Calls: {tool_call}") for tool_call in response.tool_calls]

User: hanoi weather


2025-04-18 00:10:17.443 | INFO     | openagentkit.modules.openai.openai_executor:execute:164 - Response Received: role='assistant' content=None tool_calls=[{'id': 'call_SHBPcf7A1Jkt3ZT2j2c93MZA', 'type': 'function', 'function': {'arguments': '{"city":"Hanoi"}', 'name': 'get_weather'}}] tool_results=None refusal=None audio=None usage=UsageResponse(prompt_tokens=229, completion_tokens=16, total_tokens=245, prompt_tokens_details=PromptTokensDetails(cached_tokens=0, audio_tokens=0), completion_tokens_details=CompletionTokensDetails(reasoning_tokens=0, audio_tokens=0, accepted_prediction_tokens=0, rejected_prediction_tokens=0))
2025-04-18 00:10:17.445 | INFO     | openagentkit.core.handlers.tool_handler:handle_tool_request:179 - Tool Result: Weather in Hanoi: sunny, 20°C, feels like 22°C, humidity: 50%


AI: None
Tool Calls: {'id': 'call_SHBPcf7A1Jkt3ZT2j2c93MZA', 'type': 'function', 'function': {'arguments': '{"city":"Hanoi"}', 'name': 'get_weather'}}
AI: None


2025-04-18 00:10:18.492 | INFO     | openagentkit.modules.openai.openai_executor:execute:164 - Response Received: role='assistant' content='The current weather in Hanoi is sunny, with a temperature of 20°C (feels like 22°C) and humidity at 50%.' tool_calls=None tool_results=None refusal=None audio=None usage=UsageResponse(prompt_tokens=278, completion_tokens=31, total_tokens=309, prompt_tokens_details=PromptTokensDetails(cached_tokens=0, audio_tokens=0), completion_tokens_details=CompletionTokensDetails(reasoning_tokens=0, audio_tokens=0, accepted_prediction_tokens=0, rejected_prediction_tokens=0))


AI: The current weather in Hanoi is sunny, with a temperature of 20°C (feels like 22°C) and humidity at 50%.


In [ ]:
executor.get_history()

[{'role': 'system',
  'content': '\n    You are a helpful assistant that can answer questions and help with tasks.\n    You are also able to use tools to get information.\n    \n        Current date and time: 2025-04-18 00:09:29\n\n        \n        '},
 {'role': 'user', 'content': 'hanoi weather'},
 {'role': 'assistant',
  'tool_calls': [{'id': 'call_Yp7f7gDxgPLH0BriqpkqEtqD',
    'type': 'function',
    'function': {'arguments': '{"city":"Hanoi"}', 'name': 'get_weather'}}],
  'content': 'None'},
 ToolCallMessage(role='tool', tool_call_id='call_Yp7f7gDxgPLH0BriqpkqEtqD', content='Weather in Hanoi: sunny, 20°C, feels like 22°C, humidity: 50%'),
 {'role': 'assistant',
  'content': 'The current weather in Hanoi is sunny, with a temperature of 20°C (feels like 22°C) and humidity at 50%.'},
 {'role': 'assistant',
  'content': 'The current weather in Hanoi is sunny, with a temperature of 20°C (feels like 22°C) and humidity at 50%.'}]